# Read the data from Mongo DB loaded previously

## Opens the connection to mongoDB and extract the records loaded from previous notebook


#### Author: Rohan Indrajeet Jadhav [x20169043@student.ncirl.ie]

#### Contents

1. Extract the data from MongoDb loaded
2. Preprocessing:
    a. Check NULL values and handle it
    b. Remove un-necessary columns
    c. Renaming the columns if required
3. Basic EDA [Exploratory Data analysis]
4. Load the data into local Postgre SQl DB


In [1]:
#Importing the libraries required

import pymongo
from pprint import pprint  
import pandas as pd
import yfinance as yf
import psycopg2
import psycopg2.extras as extras
import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
# This command will open connection to mongoDB
# Read the data from DB :: covidStockAnalysis and Collection name :: stocksData
try:
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    # Database Name
    db = client["covidStockAnalysis"]
    # Collection Name
    col = db["stocksData"]
    allStockData = col.find()
except: 
    print("Error in connection or fetching data from MongoDB:")

allStockDict = {}

dfAllStocks =  pd.DataFrame(list(allStockData))

dfAllStocks

,_id,index,Date,Open,High,Low,Close,Adj Close,Volume,stockName,countryName
0,608306a59b0adc96b2c8f7d0,0,2019-11-01,3.120000,3.368000,2.921600,3.160000,3.160000,215370.0,CGIX,Austria
1,608306a59b0adc96b2c8f7d1,1,2019-11-04,3.180000,3.580000,3.120000,3.430000,3.430000,324657.0,CGIX,Austria
2,608306a59b0adc96b2c8f7d2,2,2019-11-05,3.490000,3.800000,3.270000,3.330000,3.330000,307128.0,CGIX,Austria
3,608306a59b0adc96b2c8f7d3,3,2019-11-06,3.290000,3.320000,3.020000,3.080000,3.080000,99570.0,CGIX,Austria
4,608306a59b0adc96b2c8f7d4,4,2019-11-07,3.070000,3.100000,2.860000,3.000000,3.000000,113341.0,CGIX,Austria
...,...,...,...,...,...,...,...,...,...,...,...
9405,608306bf9b0adc96b2c91ca7,366,2021-04-19,51.410000,51.459999,51.110001,51.220001,51.220001,15499900.0,VEA,European
9406,608306bf9b0adc96b2c91ca8,367,2021-04-20,50.740002,50.779999,50.279999,50.369999,50.369999,11657400.0,VEA,European
9407,608306bf9b0adc96b2c91ca9,368,2021-04-21,50.250000,50.860001,50.209999,50.830002,50.830002,11195800.0,VEA,European
9408,608306bf9b0adc96b2c91caa,369,2021-04-22,50.919998,50.959999,50.549999,50.619999,50.619999,9660600.0,VEA,European


##  Pre-processing 
### 1. Removing unwanted columns 


In [3]:
dfAllStocksSelectiveCol = dfAllStocks[["stockName", "countryName", "Date", "Open", "Close", "High", "Low", "Adj Close", "Volume"]]

### 2. Renaming the column names like 'close' to 'closeValue' as this is used as keyword


In [4]:
dfAllStocksSelectiveColNew = dfAllStocksSelectiveCol.rename(columns={"Adj Close": "AdjClose"})

finalDFToLoad = dfAllStocksSelectiveColNew.rename(columns={"Date": "stockDate"})

In [5]:
dfData = finalDFToLoad.rename(columns={"Open": "openValue"})
dfData = dfData.rename(columns={"Close": "closeValue"})

dfData

,stockName,countryName,stockDate,openValue,closeValue,High,Low,AdjClose,Volume
0,CGIX,Austria,2019-11-01,3.120000,3.160000,3.368000,2.921600,3.160000,215370.0
1,CGIX,Austria,2019-11-04,3.180000,3.430000,3.580000,3.120000,3.430000,324657.0
2,CGIX,Austria,2019-11-05,3.490000,3.330000,3.800000,3.270000,3.330000,307128.0
3,CGIX,Austria,2019-11-06,3.290000,3.080000,3.320000,3.020000,3.080000,99570.0
4,CGIX,Austria,2019-11-07,3.070000,3.000000,3.100000,2.860000,3.000000,113341.0
...,...,...,...,...,...,...,...,...,...
9405,VEA,European,2021-04-19,51.410000,51.220001,51.459999,51.110001,51.220001,15499900.0
9406,VEA,European,2021-04-20,50.740002,50.369999,50.779999,50.279999,50.369999,11657400.0
9407,VEA,European,2021-04-21,50.250000,50.830002,50.860001,50.209999,50.830002,11195800.0
9408,VEA,European,2021-04-22,50.919998,50.619999,50.959999,50.549999,50.619999,9660600.0


In [6]:
dfData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9410 entries, 0 to 9409
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   stockName    9410 non-null   object        
 1   countryName  9410 non-null   object        
 2   stockDate    9410 non-null   datetime64[ns]
 3   openValue    9409 non-null   float64       
 4   closeValue   9409 non-null   float64       
 5   High         9409 non-null   float64       
 6   Low          9409 non-null   float64       
 7   AdjClose     9409 non-null   float64       
 8   Volume       9409 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 661.8+ KB


### 3. Check NULL or blanks in dataframe and remove those

In [7]:
# Check NULL values in dataframe

dfData.isnull().sum()

stockName      0
countryName    0
stockDate      0
openValue      1
closeValue     1
High           1
Low            1
AdjClose       1
Volume         1
dtype: int64

In [8]:

dfData[dfData['High'].isnull()]

,stockName,countryName,stockDate,openValue,closeValue,High,Low,AdjClose,Volume
7554,UL,European,2019-10-31,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
#This will delete all NULL valued records from dataframe

dfData = dfData.dropna()

In [24]:
dfData.shape

(9409, 9)

### Basic EDA [Exploratory Data Analysis]

In [11]:
dfData.describe()

,openValue,closeValue,High,Low,AdjClose,Volume
count,9409.000000,9409.000000,9409.000000,9409.000000,9409.000000,9.409000e+03
mean,130.421917,130.427528,131.079584,129.742420,129.773231,1.801642e+06
std,798.700512,798.704504,798.758561,798.639891,798.744485,4.084461e+06
min,0.000010,0.000010,0.000010,0.000010,0.000010,0.000000e+00
25%,5.480000,5.460000,5.590000,5.360000,5.300914,7.000000e+03
50%,35.730000,35.680000,36.080002,35.279999,34.075001,7.326000e+05
75%,82.739998,82.389999,83.620003,81.019997,80.000000,1.803400e+06
max,10219.799805,10219.799805,10219.799805,10219.799805,10219.799805,1.098105e+08


### Statistical analysis for showing correlation between the fields 

In [12]:
dfData.corr()

,openValue,closeValue,High,Low,AdjClose,Volume
openValue,1.000000,0.999997,0.999997,0.999997,0.999996,-0.049730
closeValue,0.999997,1.000000,0.999998,0.999997,0.999999,-0.049751
High,0.999997,0.999998,1.000000,0.999994,0.999997,-0.049834
Low,0.999997,0.999997,0.999994,1.000000,0.999997,-0.049631
AdjClose,0.999996,0.999999,0.999997,0.999997,1.000000,-0.049789
Volume,-0.049730,-0.049751,-0.049834,-0.049631,-0.049789,1.000000


In [13]:
# Showing unique country names from extarcted records
dfData.countryName.unique()

array(['Austria', 'Belgium', 'Ireland', 'Italy', 'Netherland', 'Spain',
       'Germany', 'Romania', 'France', 'European'], dtype=object)

In [14]:
# Showing unique stock names from extarcted records
dfData.stockName.unique()

array(['CGIX', 'IGH', 'ATPL', 'ORPH', 'ISP', 'TS', 'ATL', 'ASML', 'SAN',
       'BBVA', 'TEF', 'AMS', 'DAX', 'SAP', 'ADS', 'BAYN', 'BET', 'SAF',
       'SNY', 'RDEIY', 'BTI', 'UL', 'RHHBY', 'NVS', 'NVO', 'VEA'],
      dtype=object)

In [15]:
dfData

,stockName,countryName,stockDate,openValue,closeValue,High,Low,AdjClose,Volume
0,CGIX,Austria,2019-11-01,3.120000,3.160000,3.368000,2.921600,3.160000,215370.0
1,CGIX,Austria,2019-11-04,3.180000,3.430000,3.580000,3.120000,3.430000,324657.0
2,CGIX,Austria,2019-11-05,3.490000,3.330000,3.800000,3.270000,3.330000,307128.0
3,CGIX,Austria,2019-11-06,3.290000,3.080000,3.320000,3.020000,3.080000,99570.0
4,CGIX,Austria,2019-11-07,3.070000,3.000000,3.100000,2.860000,3.000000,113341.0
...,...,...,...,...,...,...,...,...,...
9405,VEA,European,2021-04-19,51.410000,51.220001,51.459999,51.110001,51.220001,15499900.0
9406,VEA,European,2021-04-20,50.740002,50.369999,50.779999,50.279999,50.369999,11657400.0
9407,VEA,European,2021-04-21,50.250000,50.830002,50.860001,50.209999,50.830002,11195800.0
9408,VEA,European,2021-04-22,50.919998,50.619999,50.959999,50.549999,50.619999,9660600.0


In [16]:
dfData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9409 entries, 0 to 9409
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   stockName    9409 non-null   object        
 1   countryName  9409 non-null   object        
 2   stockDate    9409 non-null   datetime64[ns]
 3   openValue    9409 non-null   float64       
 4   closeValue   9409 non-null   float64       
 5   High         9409 non-null   float64       
 6   Low          9409 non-null   float64       
 7   AdjClose     9409 non-null   float64       
 8   Volume       9409 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 735.1+ KB


In [26]:
# Checking the connection with local PostgreSQL 
try:
    conn = psycopg2.connect(
        host="localhost",
        database="stockDB",
        user="postgres",
        password="Optum@2021")
    print("Connection established succesfully")
except:
    print("Error in connecting the local postgreSQL server ")

conn.autocommit = True

Connection established succesfully


In [18]:
#function created for inserting the data to Loalhosted PostgreSQL
# Three parameters required 1. connection string, dataframe, table_name

def insert_records(conn,df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    
    tuples = [tuple(x) for x in df.to_numpy()]
    
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    
    # SQL quert to execute
    truncateSQL = "truncate table {0}".format(table)
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    print("Required variables {0} :: {1} ".format(truncateSQL,query))
    
    try:
        cursor.execute(truncateSQL)
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("Records are inserted please check !!")
    cursor.close()

In [19]:
# Calling the function by passing the parameter as
# 1. connectionstring 
# 2. dataframe that need to load into locahost
# 3. Table name in which data has to be loaded 

insert_records(conn, dfData ,"stockdata")

Required variables truncate table stockdata :: INSERT INTO stockdata(stockName,countryName,stockDate,openValue,closeValue,High,Low,AdjClose,Volume) VALUES %s 
Records are inserted please check !!


In [20]:
# Calling next notebook to load the data to Azure cloud

%run ./loadDataToAzure_Rohan_3.ipynb